In [1]:
import google.generativeai as genai
import pandas as pd

In [2]:
def get_api_key(file_path):
    with open(file_path, "r") as file:
        return file.read().strip()

api_key_file = "./API_Key/Gemini.txt"  
api_key = get_api_key(api_key_file)
genai.configure(api_key=api_key)

model = genai.GenerativeModel("models/gemini-1.5-flash")

In [3]:
def configure_genai(api_key_file):
    with open(api_key_file, "r") as file:
        api_key = file.read().strip()
    genai.configure(api_key=api_key)
    return genai.GenerativeModel("gemini-1.5-flash")  

In [4]:
def generate_ner_tags(sentence, model):
    prompt = f"""
    You are a business process modelling expert, tasked with identifying which mentions of certain process relevant elements in textual descriptions of business processes belong to the same entity. 

    Allowed NER tags and their explanations:
    - O: An O tag indicates that a token belongs to no chunk.
    - B-Actor: This tag indicates the beginning of an Actor chunk.
    - I-Actor: This tag indicates that the token is inside an Actor chunk.
    - B-Activity: This tag indicates the beginning of an Activity chunk.
    - I-Activity: This tag indicates that the token is inside an Activity chunk.
    - B-Activity Data: This tag indicates the beginning of an Activity Data chunk.
    - I-Activity Data: This tag indicates that the token is inside an Activity Data chunk.
    - B-Further Specification: This tag indicates the beginning of a Further Specification chunk.
    - I-Further Specification: This tag indicates that the token is inside a Further Specification chunk.
    - B-XOR Gateway: This tag indicates the beginning of a XOR Gateway chunk.
    - I-XOR Gateway: This tag indicates that the token is inside a XOR Gateway chunk.
    - B-Condition Specification: This tag indicates the beginning of a Condition Specification chunk.
    - I-Condition Specification: This tag indicates that the token is inside a Condition Specification chunk.
    - B-AND Gateway: This tag indicates the beginning of an AND Gateway chunk.
    - I-AND Gateway: This tag indicates that the token is inside an AND Gateway chunk.

    An example of this could look like this:
    Sentence:
    ['The', 'post', 'is', 'advertised', ',', 'applicants', 'apply', ',', 'the', 'applications', 'are', 'checked', 'and', 'the', 'post', 'is', 'filled', '.']
    NER_tags:
    ['B-Activity Data', 'I-Activity Data', 'O', 'B-Activity', 'O', 'B-Actor', 'B-Activity', 'O', 'B-Activity Data', 'I-Activity Data', 'O', 'B-Activity', 'O', 'B-Activity Data', 'I-Activity Data', 'O', 'B-Activity', 'O']
    
    
    Now, label each token in the following sentence with one of the above NER tags:
    
    Sentence: {sentence}
    
    Output ONLY a Python-style list of NER tags. Do not include any explanations or additional text.
    """
    response = model.generate_content(prompt)
    return response.text.strip()

In [5]:
def process_dataset(input_file, output_file, api_key_file):
    model = configure_genai(api_key_file)
    if model is None:
        raise ValueError("Failed to initialize the Gemini model. Check the API key and model name.")

    dataset = pd.read_csv(input_file)

    def process_row(row):
        sentence = " ".join(eval(row))  
        try:
            return generate_ner_tags(sentence, model)
        except Exception as e:
            return f"Error: {str(e)}"

    dataset["Gemini-1.5-flash"] = dataset["tokens"].apply(process_row)

    dataset.to_csv(output_file, index=False)
    print(f"Updated dataset saved to {output_file}")

In [6]:
input_file = "PET-Customized.csv" 
output_file = "PET-Customized-gemini-1.5-flash.csv" 
api_key_file = "./API_Key/Gemini.txt"

process_dataset(input_file, output_file, api_key_file)

Updated dataset saved to PET-Customized-gemini-1.5-flash.csv
